In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from schema import Schema
from umi_surrogate import UmiSurrogate
from surrogate import Surrogate

c:\Users\zoele\anaconda3\envs\ml-for-bem\lib\site-packages\outdated\utils.py:14: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  return warn(


Couldn't import dot_parser, loading of dot files will not be possible.


: 

: 

In [ ]:
# surrogate = Surrogate(schema=schema)

# print(surrogate.eui_core_heating_max)
# print(surrogate.eui_core_cooling_max)

In [ ]:
# umi_path = "D:/Users/zoelh/GitRepos/ml-for-building-energy-modeling/umi/Sample/SampleBuildings.umi"
# energy_results_path = "C:/Users/zoele/Git_Repos/ml-for-building-energy-modeling/umi/SampleBuildings/eplus"

umi_path = "C:/Users/zoele/Dropbox (MIT)/Thesis Work/Florianopolis/Florianopolis_reference2004energy.umi"
energy_results_path = "C:/Users/zoele/Dropbox (MIT)/Thesis Work/Florianopolis/energy/Florianopolis_reference2004/eplus"

schema = Schema()

Load and open the umi file - this may take a few moments.

In [ ]:
umi = UmiSurrogate.open(
    umi_path=umi_path, 
    schema=schema,
    compute_loss=True,
    energy_results_path=energy_results_path,
    checkpoint="deeper_output_net_with_leaky_relus_resumed/deeper_output_net_with_leaky_relus_resumed_202307232139_000_350000.pt")

In [ ]:
# pandas_df = pd.read_csv("C:/Users/zoele/Dropbox (MIT)/Thesis Work/Florianopolis/energy/Florianopolis_reference2004/eplus/Group0/SAMPLE0/40/SAMPLE0.csv")
# pandas_df.columns = pandas_df.columns.str.strip()

In [ ]:
# new_p = "D:/Users/zoelh/GitRepos/ml-for-building-energy-modeling/umi/Sample/SampleBuildings/eplus"
# new_p = "C:/Users/zoele/Git_Repos/ml-for-building-energy-modeling/umi/SampleBuildings/eplus"
# umi.set_energy_path(new_p)
umi.fetch_raw_shoebox_results(override=False)
print(umi.shoeboxdf.shape)
umi.shoeboxdf.head()

In [ ]:
umi.shoeboxdf.columns

In [ ]:
idf_path = umi.shoeboxdf['ShoeboxPath'][0]
csv_path = idf_path.replace("idf", "csv")
pandas_df = pd.read_csv(csv_path)
pandas_df.filter(regex="PERIM").columns

Temporary fix of window values

In [ ]:
for t in umi.templates:
    t_vect = umi.template_vectors_dict[t]['template_vect']
    # replace u_value if too high
    if t_vect[-3] > 1:
        print("way too high u_val")
        t_vect[-3] = 1.0

## Surrogate

In [ ]:
true_loads, pred_loads, all_losses = umi.run_umi_batch(count=1000, batch_size=5)

# TODO: something takes a long time after building the dataloader

## Check shoebox ratios

Ratio distributions of the umi file

In [ ]:
umi.shoeboxdf.columns

In [ ]:
ratios = ['Roof2FloorRatio', 'Ground2FloorRatio', 'Floor2Fac', 'Core2Perimeter']
fig, axs = plt.subplots(1, 4, figsize=(10, 3))

for i, ratio in enumerate(ratios):
    umi.shoeboxdf[ratio].hist(ax=axs[i])
    print(f"{ratio} min: {umi.shoeboxdf[ratio].min()}, max {umi.shoeboxdf[ratio].max()}")
    axs[i].set_xlabel(ratio)
fig.show()

Area and ratio calcs check

In [ ]:
%%time
# WARNING: THIS TAKES A WHILE
shoebox_collection = umi._fetch_shoeboxcollection()

In [ ]:
core_areas = []
perim_areas = []
areas = []
widths = []
lengths = []

for name, sb in shoebox_collection.items():
    areas.append(sb.total_building_area)
    widths.append(sb.width)
    lengths.append(sb.length)
    perim_areas.append(sb.idfobjects["ZONE"][0].Floor_Area)
    core_areas.append(sb.idfobjects["ZONE"][1].Floor_Area)

In [ ]:
shoebox_collection[2].view_model()

In [ ]:
surfs = []
for surf in shoebox_collection[2].idfobjects["BUILDINGSURFACE:DETAILED"]:
    if surf.Surface_Type.upper() == "FLOOR":
        surfs.append(surf)
calc_a = 0
for s, surf in enumerate(surfs):
    print(surf.area)
    calc_a += surf.area
    if s == 0:
        x = float(getattr(surf, f"Vertex_{1}_Xcoordinate"))
        y = float(getattr(surf, f"Vertex_{1}_Ycoordinate"))
        v1 = v2 = v3 = v4 = (x, y)
    for i in range(2, 5):
        x = float(getattr(surf, f"Vertex_{i}_Xcoordinate"))
        y = float(getattr(surf, f"Vertex_{i}_Ycoordinate"))
        if x > v1[0]:
            v1 = (x, y)
        if x < v2[0]:
            v2 = (x, y)
        if y > v3[1]:
            v3 = (x, y)
        if y < v4[1]:
            v4 = (x, y)
            
print(v1, v2, v3, v4)
vertices = np.array([v1, v2, v3, v4])
plt.scatter(vertices[:, 0], vertices[:, 1])
# print(f"Area = {x_max-x_min}")

In [ ]:
idx = 25
print("Fractions calculated length", (umi.shoeboxdf["Core2Perimeter"][idx] + 1) * umi.perim_offset)
print("Geomeppy calculated length", shoebox_collection[idx].length)
print("Geomeppy calculated width", shoebox_collection[idx].width)
print("Fractions calculated area", (umi.shoeboxdf["Core2Perimeter"][idx] + 1) * umi.perim_offset * umi.width)
print("Geomeppy-based area (lxw)", shoebox_collection[idx].width*shoebox_collection[idx].length)
print("EPlus calculated area by zone area", 
    shoebox_collection[idx].idfobjects["ZONE"][0].Floor_Area + 
    shoebox_collection[idx].idfobjects["ZONE"][1].Floor_Area
    )
print("Archetypal calculated area", shoebox_collection[idx].total_building_area)
print("Inputs (fractions) calculated perim area", (umi.perim_offset * umi.width))
print("Fractions calculated perim area (with sb.total_building_area)", shoebox_collection[idx].total_building_area/(umi.shoeboxdf["Core2Perimeter"][idx]+1))
print("Eplus calculated perim area", shoebox_collection[idx].idfobjects["ZONE"][0].Floor_Area)

In [ ]:
for surf in shoebox_collection[idx].idfobjects["ZONE"][1].zonesurfaces:
    try:
        print(surf.Surface_Type)
        print(surf.tilt)
        print(surf.area)
    except:
        pass

In [ ]:
print(max(areas))
print(min(areas))

In [ ]:
print(max(widths))
print(min(widths))
print(max(lengths))
print(min(lengths))

In [ ]:
print(max(core_areas))
print(min(core_areas))

In [ ]:
print(max(perim_areas))
print(min(perim_areas))

## Plot the building parameter distributions

In [ ]:
umi.surrogate_plot_params(start_ix=0, count=262, include_whiskers=True, title="Parameter Overview")

Plot the results

In [ ]:
import torch
from sklearn.metrics import r2_score

def plot_model_fits(true_loads, pred_loads):
    true_loads = torch.sum(true_loads, axis=2)
    pred_loads = torch.sum(pred_loads, axis=2)
    maxes = torch.max(true_loads, dim=0)[0].reshape(1, 4)
    true_loads = true_loads / maxes
    pred_loads = pred_loads / maxes
    true_loads = true_loads.cpu()
    pred_loads = pred_loads.cpu()
    r2_scores = []
    for i, zone_name in enumerate(
        ("Perimeter Heating", "Perimeter Cooling", "Core Heating", "Core Cooling")
    ):
        r2 = r2_score(true_loads[:, i], pred_loads[:, i])
        print(zone_name, r2)
        r2_scores.append(r2)
    fig, axs = plt.subplots(2, 2, figsize=(12, 12))
    identity = np.linspace(0, 1, 10)
    plt.suptitle("Annual Model Fits")
    axs[0, 0].scatter(true_loads[:, 0], pred_loads[:, 0], s=1, alpha=0.3)
    axs[0, 1].scatter(true_loads[:, 1], pred_loads[:, 1], s=1, alpha=0.3)
    axs[1, 0].scatter(true_loads[:, 2], pred_loads[:, 2], s=1, alpha=0.3)
    axs[1, 1].scatter(true_loads[:, 3], pred_loads[:, 3], s=1, alpha=0.3)
    axs[0, 0].plot(identity, identity, color="dodgerblue", label="Perfect Model")
    axs[0, 1].plot(identity, identity, color="dodgerblue")
    axs[1, 0].plot(identity, identity, color="dodgerblue")
    axs[1, 1].plot(identity, identity, color="dodgerblue")
    axs[0, 0].set_ylabel("Perimeter")
    axs[1, 0].set_ylabel("Core")
    axs[1, 0].set_xlabel("Heating")
    axs[1, 1].set_xlabel("Cooling")
    axs[0, 0].legend()
    fig.tight_layout()

In [ ]:
true_loads.max()

In [ ]:
pred_loads.max()

In [ ]:
plot_model_fits(true_loads, pred_loads)

In [ ]:
# ix = np.random.choice(np.arange(true_loads.shape[0]), plot_count, replace=False)
fig, axs = plt.subplots(1, 2, figsize=(10, 4))
plt.suptitle("Model Comparison")

lineweight = 0.1
alpha = 0.5

for i in range(1262):
    axs[0].plot(true_loads[i, 0].cpu(), label="Heating (true)", linewidth=lineweight, alpha=alpha, color='orange')
    axs[0].plot(true_loads[i, 1].cpu(), label="Cooling (true)", linewidth=lineweight, alpha=alpha, color='cornflowerblue')
    axs[1].plot(true_loads[i, 2].cpu(), label="Heating (true)", linewidth=lineweight, alpha=alpha, color='orange')
    axs[1].plot(true_loads[i, 3].cpu(), label="Cooling (true)", linewidth=lineweight, alpha=alpha, color='cornflowerblue')
    # axs[0].plot(pred_loads[i, 0].cpu(), "-o", label="Heating (predicted)")
    # axs[0].plot(pred_loads[i, 1].cpu(), "-o", label="Cooling (predicted)")
    # axs[1].plot(pred_loads[i, 2].cpu(), "-o", label="Heating (predicted)")
    # axs[1].plot(pred_loads[i, 3].cpu(), "-o", label="Cooling (predicted)")

axs[0].set_ylim([-0.01, 1])
axs[1].set_ylim([-0.01, 10])
axs[0].set_title("Perimeter")
axs[1].set_title("Core")
fig.show()


Monthly comparison

In [ ]:
def plot_model_comparisons(true_loads, pred_loads, plot_count=4, ylim=[-0.01, 1]):
    ix = np.random.choice(np.arange(true_loads.shape[0]), plot_count, replace=False)
    for i in ix:
        fig, axs = plt.subplots(1, 2, figsize=(10, 4))
        plt.suptitle("Model Comparison")
        axs[0].plot(true_loads[i, 0].cpu(), label="Heating (true)")
        axs[0].plot(true_loads[i, 1].cpu(), label="Cooling (true)")
        axs[1].plot(true_loads[i, 2].cpu(), label="Heating (true)")
        axs[1].plot(true_loads[i, 3].cpu(), label="Cooling (true)")
        axs[0].plot(pred_loads[i, 0].cpu(), "-o", label="Heating (predicted)")
        axs[0].plot(pred_loads[i, 1].cpu(), "-o", label="Cooling (predicted)")
        axs[1].plot(pred_loads[i, 2].cpu(), "-o", label="Heating (predicted)")
        axs[1].plot(pred_loads[i, 3].cpu(), "-o", label="Cooling (predicted)")
        # axs[0].set_ylim(ylim)
        # axs[1].set_ylim(ylim)
        axs[0].set_title("Perimeter")
        axs[1].set_title("Core")
        axs[0].legend()
        axs[1].legend()


In [ ]:
plot_model_comparisons(true_loads, pred_loads, plot_count=3, ylim=[-0.01,8])

Plot the errors/losses

In [ ]:
print(all_losses.mean())
print(all_losses.min())
print(all_losses.max())

## Visualizations

In [ ]:
# Plot all of the radiation values

import matplotlib.pyplot as plt
from pandas import DatetimeIndex

orient_lookup = {
    'North': 0,
    'East': 1,
    'South': 2,
    'West': 3
}
cols = ['red', 'blue', 'orange', 'purple']

shoebox_ids = umi.get_shoebox_template_ids(0, 364)

fig, axes = plt.subplots(2, 2, figsize=(10, 8))
axes = axes.flatten()
for sb_id in shoebox_ids:
    orient = umi.shoeboxdf['Orientation'][sb_id] #.to_list()[0]
    idf_path = umi.shoeboxdf['ShoeboxPath'][sb_id]
    csv_path = idf_path.replace("idf", "csv")
    df = pd.read_csv(csv_path)
    rad = df['PERIM:Zone Windows Total Transmitted Solar Radiation Energy [J](Hourly)']
    rad = df.groupby(np.arange(len(rad))//24).sum()
    axes[orient_lookup[orient]].plot(rad, cols[orient_lookup[orient]],alpha=0.1)
for i, ax in enumerate(axes):
    ax.set_title(list(orient_lookup.keys())[i])
fig.suptitle("Solar gain through window by orientation")
fig.tight_layout()
plt.show()